In [ ]:
import numpy as np
import pandas as pd
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from sklearn.model_selection import train_test_split
from PIL import Image
from skimage.morphology import binary_opening, disk, label
import os
import matplotlib.pyplot as plt
import sys
import gc
import time
from keras import backend as K
import tensorflow as tf
import keras
print(os.listdir("../input"))

print('start!!')

param={
    #model training relevent
    'samplesize':2000,# use how many training samples to train
    'validatesize':2,#how many samples to use for validation 
    'bufferSize':200,# load how many training images from disk to memory at a time
    'batchSize':20,# train_on_batch batch size
    'epoch':1,#how many epochs to train
    
    #submission relevent
    'test':True,#if true ,only predict and submit a number of test images specified by 'testSize'
    'testSize':10,#number of test images for prediction and submission
    'subBatchSize':200#predict and submit a batch of test samples at a time(limited memory)
}


In [ ]:
def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction

In [ ]:
def multi_rle_encode(img, **kwargs):
    '''
    Encode connected regions as separated masks
    '''
    labels = label(img)
    if img.ndim > 2:
        return [rle_encode(np.sum(labels==k, axis=2), **kwargs) for k in np.unique(labels[labels>0])]
    else:
        return [rle_encode(labels==k, **kwargs) for k in np.unique(labels[labels>0])]
    
    
# ref: https://www.kaggle.com/paulorzp/run-length-encode-and-decode
def rle_encode(img, min_max_threshold=1e-3, max_mean_threshold=None):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    if np.max(img) < min_max_threshold:
        return '' ## no need to encode if it's all zeros
    if max_mean_threshold and np.mean(img) > max_mean_threshold:
        return '' ## ignore overfilled mask
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
def imageToArray(filepath):
    arr=np.array(Image.open(filepath),dtype=np.uint8,copy=False)
    return arr


In [ ]:
os.chdir(r'../input/')

In [ ]:
mask_train=pd.read_csv('train_ship_segmentations.csv')

In [ ]:
ship=mask_train.EncodedPixels.apply(lambda x:isinstance(x,str)+0)
tmp=mask_train.drop('EncodedPixels',axis=1)
tmp['ship']=ship

In [ ]:
unique_id=tmp.groupby(by='ImageId').agg(sum)
(unique_id.ship>0).sum()
(unique_id.ship==0).sum()

downsize_unique=unique_id.query('ship!=0').sample(param['samplesize'])

# 1% validate sample
id_train,id_validate=train_test_split(downsize_unique.index.values,test_size=param['validatesize'])

mask_train=mask_train.set_index('ImageId')

downsize_mask_train=mask_train.loc[id_train]

downsize_mask_validate=mask_train.loc[id_validate]

In [ ]:
 # helper class to retrieve train image and train masks one batch at a time
 # we can use the getNextBatch function to retrieve train data in batches and feed them to keras train_on_batch function
class BatchWrapper:
    bufferIx = 0
    innerBufferIx = 0
    image_train = []  # buffer
    mask_train = []  # buffer

    def __init__(self, masks, batchSize=param['batchSize'], bufferSize=param['bufferSize']):
        self.masks = masks
        self.batchSize = batchSize
        self.bufferSize = bufferSize

    def hasNext(self):
        return self.bufferIx <= (len(self.masks) - 1) or self.innerBufferIx <= (len(self.image_train) - 1)

    def getNextBatch(self):
        if self.innerBufferIx >= len(self.image_train):
            self.getNextBuffer()
            self.innerBufferIx = 0
        aa = self.image_train[self.innerBufferIx:self.innerBufferIx + self.batchSize]
        bb = self.mask_train[self.innerBufferIx:self.innerBufferIx + self.batchSize]
        self.innerBufferIx += self.batchSize
        return (aa, bb)

    def getNextBuffer(self):
        if self.bufferIx >= len(self.masks):
            self.bufferIx = 0
        tmp = self.masks.iloc[self.bufferIx:self.bufferIx + self.bufferSize]
        self.getImage(tmp)
        self.bufferIx += self.bufferSize

    def getImage(self, batch_masks):
        self.image_train = []
        self.mask_train = []
        for ix, id in enumerate(batch_masks.index):
            self.image_train.append(imageToArray(os.path.join(os.curdir, 'train', id)))
            if isinstance(self.masks.EncodedPixels.loc[id], str):
                self.mask_train.append(rle_decode(self.masks.EncodedPixels.loc[id]).reshape(768, 768, 1))
            else:
                self.mask_train.append(np.zeros((768, 768, 1), dtype=np.uint8))
        self.image_train = np.array(self.image_train, dtype=np.float32, copy=False) / 255
        self.mask_train = np.array(self.mask_train)
        

In [ ]:
train_wrapper=BatchWrapper(masks=downsize_mask_train)
validate_wrapper=BatchWrapper(masks=downsize_mask_validate)
image_validate,mask_validate=validate_wrapper.getNextBatch()


In [ ]:
def ioumetric(y_true,y_pred):
    intersection = K.sum(y_true * y_pred)
    sum_ = K.sum(y_true + y_pred)
    jac = intersection / (sum_ - intersection)
    return jac

def ioumetric2(y_true,y_pred):
    sum=0
    for g,p in zip(y_true,y_pred):
        intersection = np.sum(y_true * y_pred)
        sum_ = np.sum(y_true + y_pred)
        jac = intersection / (sum_ - intersection)
        sum+=jac
    return sum/len(y_true)

In [ ]:
def iouloss(y_true,y_pred):
    intersection = K.sum(y_true * y_pred)
    sum_ = K.sum(y_true + y_pred)
    jac = (intersection) / (sum_ - intersection)
    return 1-jac

In [ ]:
# def focal_crossentropy(y_true,y_pred):
#     t1=K.binary_crossentropy(y_true, y_pred)
#     t2=tf.where(tf.equal(y_true,0),t1*(y_pred**10),t1*((1-y_pred)**10))
#     return t2
def focal_crossentropy(y_true,y_pred):
    #_epsilon = _to_tensor(epsilon(), y_pred.dtype.base_dtype)
    t1=-y_true*K.log(y_pred)*((1-y_pred)**8)-(1-y_true)*K.log(1-y_pred)*(y_pred**8)
    return t1

In [ ]:
# def biased_crossentropy(y_true,y_pred):
#     t1=K.binary_crossentropy(y_true, y_pred)
#     t2=tf.where(tf.equal(y_true,0),t1*1000,t1)
#     return t2
def biased_crossentropy(y_true,y_pred):
    t1=-y_true*K.log(y_pred)*1000-(1-y_true)*K.log(1-y_pred)
    return t1

In [ ]:
def focal_loss(gamma=8., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

In [ ]:
visual_test_img=np.array([imageToArray(os.path.join(os.curdir,'train','16148344e.jpg'))])/255
visual_test_real=rle_decode(mask_train['EncodedPixels']['16148344e.jpg']).reshape((768,768))

In [ ]:
input_image=Input(shape=(768,768,3))
c1=Conv2D(filters=8,kernel_size=(3,3),padding='same',activation='relu')(input_image)
c2=Conv2D(filters=8,kernel_size=(3,3),padding='same',activation='relu')(c1)
c3=MaxPooling2D(pool_size=(2, 2))(c2)
c21=SpatialDropout2D(rate=0.25)(c3)
c22=Conv2D(filters=16,kernel_size=(3,3),padding='same',activation='relu')(c21)
c23=Conv2D(filters=16,kernel_size=(3,3),padding='same',activation='relu')(c22)
c24=MaxPooling2D(pool_size=(2, 2))(c23)
c60=SpatialDropout2D(rate=0.25)(c24)
c61=Conv2D(filters=32,kernel_size=(3,3),padding='same',activation='relu')(c60)
c62=Conv2D(filters=32,kernel_size=(3,3),padding='same',activation='relu')(c61)
c63=MaxPooling2D(pool_size=(2, 2))(c62)
c81=SpatialDropout2D(rate=0.25)(c63)
c82=Conv2D(filters=64,kernel_size=(3,3),padding='same',activation='relu')(c81)
c83=UpSampling2D(size=(2,2))(c82)

c100=Concatenate(axis=3)([c83,c62])
c101=SpatialDropout2D(rate=0.25)(c100)
c102=Conv2D(filters=32,kernel_size=(3,3),padding='same',activation='relu')(c101)
c104=Conv2D(filters=32,kernel_size=(3,3),padding='same',activation='relu')(c102)
c105=UpSampling2D(size=(2,2))(c104)

c121=Concatenate(axis=3)([c105,c23])
c122=SpatialDropout2D(rate=0.25)(c121)
c123=Conv2D(filters=16,kernel_size=(3,3),padding='same',activation='relu')(c122)
c124=Conv2D(filters=16,kernel_size=(3,3),padding='same',activation='relu')(c123)
c125=UpSampling2D(size=(2,2))(c124)

c141=Concatenate(axis=3)([c125,c2])
c142=SpatialDropout2D(rate=0.25)(c141)
c143=Conv2D(filters=8,kernel_size=(3,3),padding='same',activation='relu')(c142)
c144=Conv2D(filters=8,kernel_size=(3,3),padding='same',activation='relu')(c143)
c145=Conv2D(filters=1,kernel_size=(3,3),padding='same',activation='sigmoid')(c144)

unet=Model(inputs=input_image,outputs=c145)
unet.compile(loss=iouloss, optimizer="adam", metrics=[ioumetric])
unet.summary()
loss_track=[]
min_loss=100
for ite in range(param['epoch']):
    print('');print('epoch:'+str(ite+1))
    loss_epoch=[]
    while train_wrapper.hasNext():
        image_batch,mask_batch=train_wrapper.getNextBatch()
        loss=unet.train_on_batch(x=image_batch,y=mask_batch)#insufficient memory
        loss_epoch.append(loss[0])
        predict_mask_validate=unet.predict(x=image_validate)
        ioumetric_validate=ioumetric2(mask_validate,predict_mask_validate)
        print("loss_train: %s iou_train: %s iou_validate: %s"%(loss[0],loss[1],ioumetric_validate))
        visual_test_mask=unet.predict(x=visual_test_img)
        #print(visual_test_mask.sum())
        fig,ax=plt.subplots(1,3)
        ax[0].imshow(visual_test_mask[0].squeeze(),cmap='gray')
        ax[0].set_title('probability image')
        ax[1].imshow(np.round(visual_test_mask[0].squeeze()),cmap='gray')
        ax[1].set_title('mask image')
        ax[2].imshow(visual_test_real,cmap='gray')
        ax[2].set_title('real mask')
        plt.show()
        del image_batch,mask_batch
        gc.collect()
    loss_track.append(loss_epoch)
    if loss_epoch[-1]<min_loss:#ModelCheckpoint
        unet.save(filepath='../working/model.h5')
        min_loss=loss_epoch[-1]
    if len(loss_track)>1 and loss_epoch[-1]>=loss_track[-2][-1]:#EarlyStopping
        break


In [ ]:
sub_id=pd.read_csv('sample_submission.csv').ImageId
if(param['test']):
    sub_id=sub_id.sample(param['testSize'])


In [ ]:
print('predicting!!')
submission=pd.DataFrame(columns=['ImageId','EncodedPixels'])
submission.to_csv('../working/mysubmission.csv',index=False,index_label=False)
sub_ix=0
while(sub_ix<len(sub_id)):
    gc.collect()
    aa=sub_id.iloc[sub_ix:sub_ix+param['subBatchSize']]
    image_test=[]
    for id in aa:
        image_test.append(imageToArray(os.path.join(os.curdir,'test',id)))
    image_test=np.array(image_test,dtype=np.float32,copy=False)/255
    mask_test=unet.predict(x=image_test)
    #print("mask pixels predict: %s"%((mask_test>0.5).sum()),end=' ')
    
    col1=[]
    col2=[]
    for ix,img in enumerate((mask_test>0.5).astype(np.uint8)):
        enc=multi_rle_encode(img)
        if len(enc)==0:
            enc=['']
        for rle_str in enc:
            col1.append(aa.iloc[ix])
            col2.append(rle_str)
    
    tp=pd.DataFrame({'ImageId':col1,'EncodedPixels':col2})
    sub=pd.read_csv(r'../working/mysubmission.csv')
    sub=pd.concat([sub,tp],axis=0)
    #print(len(sub.query('EncodedPixels==EncodedPixels')))
    sub.to_csv('../working/mysubmission.csv',index=False,index_label=False)
    sub_ix+=param['subBatchSize']
    del image_test,mask_test
    
print('done')
    